In [1]:
!java --version

openjdk 17.0.16 2025-07-15
OpenJDK Runtime Environment (build 17.0.16+8-Ubuntu-0ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 17.0.16+8-Ubuntu-0ubuntu122.04.1, mixed mode, sharing)


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("HDFS_Test") \
    .master("spark://spark:7077") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:8020") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/01 13:18:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df = spark.read.csv("hdfs://namenode:8020/data/test.csv", header=True, inferSchema=True)
df.show()

+--------------------+-----------------+--------------------+--------------------+---------+--------+
|                name|      phoneNumber|               email|             address|userAgent|hexcolor|
+--------------------+-----------------+--------------------+--------------------+---------+--------+
|Prof. Lesly Doyle IV|     265-758-1402|fannie65@hotmail.com|29603 Dickens Mot...|     NULL|    NULL|
|     Port Ivorymouth|   LA 92479-1857"|Mozilla/5.0 (comp...|             #337504|     NULL|    NULL|
|       Nat Schneider|  +1.758.270.1301|eldora.wisoky@hot...|955 Wintheiser Is...|     NULL|    NULL|
|  South Gillianburgh|   OR 19285-6036"|Mozilla/5.0 (comp...|             #d951ff|     NULL|    NULL|
|      Mrs. Loma Wiza|   (323) 980-5038|stephania.rowe@ma...|3885 Huels Loop S...|     NULL|    NULL|
|     Port Idellhaven|        TX 10368"|Mozilla/5.0 (Wind...|             #e59ab0|     NULL|    NULL|
|    Mitchell Treutel|     274-291-8107|yheathcote@dougla...|2800 Jace Motorwa...|

In [10]:
rdd = df.rdd.flatMap(lambda row: [row] * 1000)
df_big = rdd.toDF(df.schema)

In [11]:
df_big = df_big.repartition(200)

In [12]:
df_big.write.mode("overwrite").csv("hdfs://namenode:8020/data/test_big", header=True)

25/10/01 13:24:14 WARN TaskSetManager: Lost task 133.0 in stage 17.0 (TID 761) (172.18.0.8 executor 0): FetchFailed(BlockManagerId(0, 172.18.0.8, 42125, None), shuffleId=1, mapIndex=0, mapId=627, reduceId=133, message=
org.apache.spark.shuffle.FetchFailedException: Block shuffle_1_627_133 is corrupted due to DISK_ISSUE
	at org.apache.spark.errors.SparkCoreErrors$.fetchFailedError(SparkCoreErrors.scala:437)
	at org.apache.spark.storage.ShuffleBlockFetcherIterator.throwFetchFailedException(ShuffleBlockFetcherIterator.scala:1233)
	at org.apache.spark.storage.BufferReleasingInputStream.tryOrFetchFailedException(ShuffleBlockFetcherIterator.scala:1392)
	at org.apache.spark.storage.BufferReleasingInputStream.read(ShuffleBlockFetcherIterator.scala:1371)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:244)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:284)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:343)
	at java.bas

In [8]:
df1 = spark.read.csv("hdfs://namenode:8020/data/test_big", header=True, inferSchema=True).show()
df1.show()

25/10/01 13:21:27 WARN TaskSetManager: Lost task 1.0 in stage 11.0 (TID 616) (172.18.0.8 executor 0): org.apache.hadoop.fs.ChecksumException: Checksum CRC32C not matched for file /data/test_big/part-00030-6ad62b92-e313-47fe-b818-56ce405d8ab5-c000.csv at position 1191936: expected=583C4773 but computed=F61E421E, algorithm=CRC32C
	at org.apache.hadoop.util.DataChecksum.throwChecksumException(DataChecksum.java:479)
	at org.apache.hadoop.util.DataChecksum.verifyChunked(DataChecksum.java:408)
	at org.apache.hadoop.util.DataChecksum.verifyChunkedSums(DataChecksum.java:383)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderRemote.readNextPacket(BlockReaderRemote.java:216)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderRemote.read(BlockReaderRemote.java:146)
	at org.apache.hadoop.hdfs.ByteArrayStrategy.readFromBlock(ReaderStrategy.java:118)
	at org.apache.hadoop.hdfs.DFSInputStream.readBuffer(DFSInputStream.java:828)
	at org.apache.hadoop.hdfs.DFSInputStream.readWithStrategy(DFSInputStream.

Py4JJavaError: An error occurred while calling o61.csv.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 11.0 failed 4 times, most recent failure: Lost task 1.3 in stage 11.0 (TID 622) (172.18.0.8 executor 0): org.apache.hadoop.fs.ChecksumException: Checksum CRC32C not matched for file /data/test_big/part-00030-6ad62b92-e313-47fe-b818-56ce405d8ab5-c000.csv at position 1191936: expected=583C4773 but computed=7A9C95C3, algorithm=CRC32C
	at org.apache.hadoop.util.DataChecksum.throwChecksumException(DataChecksum.java:479)
	at org.apache.hadoop.util.DataChecksum.verifyChunked(DataChecksum.java:408)
	at org.apache.hadoop.util.DataChecksum.verifyChunkedSums(DataChecksum.java:383)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderRemote.readNextPacket(BlockReaderRemote.java:216)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderRemote.read(BlockReaderRemote.java:146)
	at org.apache.hadoop.hdfs.ByteArrayStrategy.readFromBlock(ReaderStrategy.java:118)
	at org.